In [1]:
%%time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary

CPU times: total: 1.66 s
Wall time: 1.73 s


In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [3]:
batch_size = 128

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True)

# Some Notes on our naive model

We are going to write a network based on what we have learnt so far.

The size of the input image is 28x28x1. We are going to add as many layers as required to reach RF = 32 "atleast".

In [4]:
class FirstDNN(nn.Module):
  def __init__(self):
    super(FirstDNN, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
    self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
    self.pool1 = nn.MaxPool2d(2, 2)
    self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
    self.conv4 = nn.Conv2d(128, 256, 3, padding = 1)
    self.pool2 = nn.MaxPool2d(2, 2)
    self.conv5 = nn.Conv2d(256, 512, 3)
    self.conv6 = nn.Conv2d(512, 1024, 3)
    self.conv7 = nn.Conv2d(1024, 10, 3)

  def forward(self, x):
    x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
    x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
    x = F.relu(self.conv6(F.relu(self.conv5(x))))

    # DON"T USE ANY NON-Linearity in Final LAYER
    x = self.conv7(x)
    x = x.view(-1, 10)
    return F.log_softmax(x)


|**NAME**| $Channel_{in}$ | $Channel_{out}$ | $r_{in}$ | $r_{out}$ | $J_{in}$ | $J_{out}$ | Pad | Stride | Kernel | Param # (bias=F)               | Param # (bias=T) |
|--------|----------------|-----------------|----------|-----------|----------|-----------|-----|--------|--------|--------------------------------|------------------|
|CONV1   | 28             |28               | 1        |3          | 1        |1          |1    |1       |3       |(3 * 3) * 32 = 288              |  320             |
|CONV2   | 28             |28               | 3        |5          | 1        |1          |1    |1       |3       |(3 * 3 * 32) * 64 = 18432       |  18496           |
|MAXPOOL | 28             |14               | 5        |6          | 1        |2          |0    |2       |2       | Nil                            |  Nil             |
|CONV3   | 14             |14               | 6        |10         | 2        |2          |1    |1       |3       |(3 * 3 * 64) * 128 = 73728      |  73,856          |
|CONV4   | 14             |14               | 10       |14         | 2        |2          |1    |1       |3       |(3 * 3 * 128) * 256 = 294912    |  295,168         |
|MAXPOOL | 14             |7                | 14       |16         | 2        |4          |0    |2       |2       |Nil                             |  Nil             |
|CONV5   | 7              |5                | 16       |24         | 4        |4          |0    |1       |3       |(3 * 3 * 256) * 512 = 1,179,648 |  1,180,160       |
|CONV6   | 5              |3                | 24       |32         | 4        |4          |0    |1       |3       |(3 * 3 * 512) * 1024 = 4,718,592|  4,719,616       |
|CONV7   | 3              |1                | 32       |40         | 4        |4          |0    |1       |3       |(3 * 3 * 1024) * 10 = 92,160    |  92,170          |

<img src='./MODEL ARCH.jpg'></img>

In [5]:
model = FirstDNN().to(device)

In [6]:
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

C:\Users\muthu\AppData\Local\Temp\ipykernel_13252\566389643.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [7]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 3):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]C:\Users\muthu\AppData\Local\Temp\ipykernel_13252\566389643.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.04729762300848961 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.60it/s] 



Test set: Average loss: 0.0557, Accuracy: 9811/10000 (98%)



loss=0.06097720190882683 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 24.84it/s]  



Test set: Average loss: 0.0391, Accuracy: 9873/10000 (99%)

